In [3]:
#import tokenization
import tensorflow as tf
import tensorflow_hub as hub
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
from bert import bert_tokenization
#from textblob.en import polarity
import numpy as np
import pandas as pd
import tensorflow as tf
#from textblob import TextBlob
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xml.etree.ElementTree as ET
#import xmltodict

# # Reading the dataset into a dataframe
# from google.colab import drive
# drive.mount('/content/drive')



tree = ET.parse('../unprocessed.tar/sorted_data/sorted_data/software/negative.review')
root = tree.getroot()

data = []
for review in root:
    review_dict = {}
    for elem in review:
        if elem.tag in ['review_text', 'rating']:
            review_dict[elem.tag] = elem.text.strip()
        review_dict["polarity"] = "negative"
    data.append(review_dict)

tree = ET.parse('../unprocessed.tar/sorted_data/sorted_data/software/positive.review')
root = tree.getroot()
for review in root:
    review_dict = {}
    for elem in review:
        if elem.tag in ['review_text', 'rating']:
            review_dict[elem.tag] = elem.text.strip()
        review_dict["polarity"] = "positive"
    data.append(review_dict)

df = pd.DataFrame(data)
df['polarity_encoded'] = df['polarity'].apply(lambda x: 1 if x == 'positive' else 0)
print(df.head())
df

   polarity rating                                        review_text  \
0  negative    2.0  The Easy Language 16 is only useful if you're ...   
1  negative    1.0  My boss asked me to get this, so that he check...   
2  negative    1.0  I found this title messing around adding thing...   
3  negative    1.0  They have the hebrew backwards!! You are suppo...   
4  negative    2.0  I just bought and installed this CD with the h...   

   polarity_encoded  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  


,polarity,rating,review_text,polarity_encoded
0,negative,2.0,The Easy Language 16 is only useful if you're ...,0
1,negative,1.0,"My boss asked me to get this, so that he check...",0
2,negative,1.0,I found this title messing around adding thing...,0
3,negative,1.0,They have the hebrew backwards!! You are suppo...,0
4,negative,2.0,I just bought and installed this CD with the h...,0
...,...,...,...,...
1910,positive,5.0,It was some years ago when I bought a previous...,1
1911,positive,5.0,The hard-copy of Britannica has been the stand...,1
1912,positive,4.0,I trust the information I find from Britannica...,1
1913,positive,4.0,If you want a simple to use program to manage ...,1


In [3]:
from sklearn.model_selection import train_test_split

# Split the data into training, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, GlobalMaxPooling1D, concatenate
from tensorflow.keras.models import Model
from transformers import BertTokenizer, TFBertModel
import re

# Preprocess the text data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 128

def preprocess(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    tokens = tokenizer.tokenize(text)
    tokens = tokens[:max_len-2]
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)
    padding = [0] * (max_len - len(input_ids))
    input_ids += padding
    input_mask += padding
    return input_ids, input_mask

# Split the data into training, validation, and test sets
x_train = [preprocess(text) for text in train_df['review_text']]
y_train = train_df['polarity_encoded']
x_val = [preprocess(text) for text in val_df['review_text']]
y_val = val_df['polarity_encoded']
x_test = [preprocess(text) for text in test_df['review_text']]
y_test = test_df['polarity_encoded']

# Load the pre-trained BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

#learning_rates = [1e-4, 1e-5, 1e-6]
#num_filters = [16, 32, 64]
optimizers = ['adam', 'SGD']
activations = ['relu', 'sigmoid']

# initialize the best score and best hyperparameters
best_score = 0
best_hyperparams = {}

# loop through all combinations of hyperparameters
for opt in optimizers:
    for act in activations:
        # build the model with the current hyperparameters
        print("Optimizer  used: ", opt, " Activation Function used : ", act)
        input_ids = Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
        input_mask = Input(shape=(max_len,), dtype=tf.int32, name='input_mask')
        embedding = bert_model(input_ids, attention_mask=input_mask)[0]
        conv1 = Conv1D(filters=32, kernel_size=3, padding='same', activation=act)(embedding)
        conv2 = Conv1D(filters=32, kernel_size=4, padding='same', activation=act)(embedding)
        conv3 = Conv1D(filters=32, kernel_size=5, padding='same', activation=act)(embedding)
        concat = concatenate([conv1, conv2, conv3])
        pooling = GlobalMaxPooling1D()(concat)
        dropout = Dropout(0.2)(pooling)
        output = Dense(1, activation='sigmoid')(dropout)
        model = Model(inputs=[input_ids, input_mask], outputs=output)

        # compile the model with the current hyperparameters

        if opt == 'adam':
            optimizer = keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
        elif opt == 'SGD':
            optimizer = keras.optimizers.SGD(learning_rate=2e-5)

        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # train the model and evaluate on the validation set
        history = model.fit(x=[np.array(x_train)[:,0], np.array(x_train)[:,1]], y=y_train,
                            validation_data=([np.array(x_val)[:,0], np.array(x_val)[:,1]], y_val),
                            epochs=3, batch_size=32, verbose=0)
        score = history.history['val_accuracy'][-1]

        # update the best score and best hyperparameters if the current model is better
        if score > best_score:
            best_score = score
            best_hyperparams = {'lr': 2e-5, 'num_filters': 32, 'optimizer': opt, 'activation': act}
            print(best_hyperparams)

# train the model with the best hyperparameters on the full training set
input_ids = Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
input_mask = Input(shape=(max_len,), dtype=tf.int32, name='input_mask')
embedding = bert_model(input_ids, attention_mask=input_mask)[0]
conv1 = Conv1D(filters=best_hyperparams['num_filters'], kernel_size=3, padding='same', activation=best_hyperparams['activation'])(embedding)
conv2 = Conv1D(filters=best_hyperparams['num_filters'], kernel_size=4, padding='same', activation=best_hyperparams['activation'])(embedding)
conv3 = Conv1D(filters=best_hyperparams['num_filters'], kernel_size=5, padding='same', activation=best_hyperparams['activation'])(embedding)
concat = concatenate([conv1, conv2, conv3])
pooling = GlobalMaxPooling1D()(concat)
dropout = Dropout(0.2)(pooling)
output = Dense(1, activation='sigmoid')(dropout)
model = Model(inputs=[input_ids, input_mask], outputs=output)

optimizer_name = best_hyperparams['optimizer']
if optimizer_name == 'adam':
    optimizer = keras.optimizers.Adam(learning_rate=lr, epsilon=1e-08)
elif optimizer_name == 'SGD':
    optimizer = keras.optimizers.SGD(learning_rate=lr)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])



# # Build the CNN model on top of the BERT model
# input_ids = Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
# input_mask = Input(shape=(max_len,), dtype=tf.int32, name='input_mask')
# embedding = bert_model(input_ids, attention_mask=input_mask)[0]
# conv1 = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(embedding)
# conv2 = Conv1D(filters=32, kernel_size=4, padding='same', activation='relu')(embedding)
# conv3 = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')(embedding)
# concat = concatenate([conv1, conv2, conv3])
# pooling = GlobalMaxPooling1D()(concat)
# dropout = Dropout(0.2)(pooling)
# output = Dense(1, activation='sigmoid')(dropout)
# model = Model(inputs=[input_ids, input_mask], outputs=output)

# Compile the model
# optimizer = keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
# model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# # Train the model
history = model.fit(x=[np.array(x_train)[:,0], np.array(x_train)[:,1]], y=y_train,
                    validation_data=([np.array(x_val)[:,0], np.array(x_val)[:,1]], y_val),
                    epochs=3, batch_size=32)

# # Evaluate the model
# model.evaluate([np.array(x_test)[:,0], np.array(x_test)[:,1]], y_test)

from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model on the test set
y_pred = model.predict([np.array(x_test)[:,0], np.array(x_test)[:,1]])
y_pred = [1 if p >= 0.5 else 0 for p in y_pred]
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Optimizer  used:  adam  Activation Function used :  relu
{'lr': 2e-05, 'num_filters': 32, 'optimizer': 'adam', 'activation': 'relu'}
Optimizer  used:  adam  Activation Function used :  sigmoid


In [10]:
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model on the test set
y_pred = model.predict([np.array(x_test)[:,0], np.array(x_test)[:,1]])
y_pred = [1 if p >= 0.5 else 0 for p in y_pred]
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

12/12 [==============================] - 76s 6s/step
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       180
           1       0.90      0.91      0.90       203

    accuracy                           0.90       383
   macro avg       0.90      0.90      0.90       383
weighted avg       0.90      0.90      0.90       383

[[159  21]
 [ 18 185]]


In [11]:
# Predict the sentiment of new review_text data
new_review_text = 'This book is amazing!'
input_ids, input_mask = preprocess(new_review_text)
test_input_pred = model.predict([np.array([input_ids]), np.array([input_mask])])
test_input_pred = ["positive" if p >= 0.5 else "negative" for p in test_input_pred]
test_input_pred

1/1 [==============================] - 0s 357ms/step


['positive']